In [2]:
!pip install pdfplumber

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for cffi: filename=cffi-1.17.1-cp38-cp38-macosx_11_0_arm64.whl size=179134 sha256=2b0de4d7f5b10641fd82ebed977e500cfb03cda4ad1e1a01b7521affce3b54b1
  Stored in directory: /Users/kwaktaekyeong/Library/Caches/pip/wheels/f8/73/f0/643f1772516a6bd314c8e57831b7d5204a45e9fe1b876b6b35
Successfully built cffi


In [20]:
import tabula.io
import json
import pandas as pd
from PyPDF2 import PdfReader
import os

def extract_tables_from_pdf(pdf_path):
    # PDF 파일에서 표를 추출
    tables = tabula.io.read_pdf(pdf_path, pages='all', multiple_tables=True, lattice=True, pandas_options={'header': None})
    return tables

def process_table_to_json(tables):
    result_dict = {}
    for table in tables:
        for index, row in table.iterrows():
            if len(row) < 3 or pd.isna(row[2]):
                continue  # 예문 또는 단어 뜻이 없거나 비어있는 행은 건너뜀
            word = row[0]
            definition = row[1]
            if pd.isna(row[2]):
                example_text = ''
                example_translation = ''
            else:
                parts = row[2].rsplit('. ', 1)
                example_text = parts[0] + '.' if len(parts) > 0 else ''
                example_translation = parts[1].strip() if len(parts) > 1 else ''

            if word not in result_dict:
                result_dict[word] = []
            entry = {
                "영어": definition,
                "난이도": 1,
                "예문": example_text,
                "예문뜻": example_translation
            }
            result_dict[word].append(entry)
    return result_dict

def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def pdf_to_json(pdf_path, json_dir):
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)
    print(f"Processing {pdf_path}, Total pages in PDF: {num_pages}")

    tables = extract_tables_from_pdf(pdf_path)
    result_dict = process_table_to_json(tables)
    
    base_name = os.path.basename(pdf_path)
    json_file_name = f"{os.path.splitext(base_name)[0]}.json"
    json_path = os.path.join(json_dir, json_file_name)
    
    save_json(result_dict, json_path)

# PDF 파일 경로와 저장할 JSON 파일이 위치할 디렉토리 경로 지정
pdf_paths = ['start.pdf', 'walk.pdf', 'jump1.pdf', 'jump2.pdf', 'run.pdf', 'fly1.pdf', 'fly2.pdf', 'master1.pdf','master2.pdf','beyond.pdf']  # 여러 PDF 파일 경로
json_dir = 'words'  # JSON 파일을 저장할 디렉토리

# 디렉토리가 없으면 생성
if not os.path.exists(json_dir):
    os.makedirs(json_dir)

# 각 PDF 파일을 별도의 JSON 파일로 변환
for pdf_path in pdf_paths:
    pdf_to_json(pdf_path, json_dir)


Processing start.pdf, Total pages in PDF: 12
Processing walk.pdf, Total pages in PDF: 12
Processing jump1.pdf, Total pages in PDF: 24
Processing jump2.pdf, Total pages in PDF: 24
Processing run.pdf, Total pages in PDF: 24
Processing fly1.pdf, Total pages in PDF: 24
Processing fly2.pdf, Total pages in PDF: 24
Processing master1.pdf, Total pages in PDF: 32
Processing master2.pdf, Total pages in PDF: 40
Processing beyond.pdf, Total pages in PDF: 24


In [41]:
import pdfplumber
import pandas as pd
import json

def pdf_to_json(pdf_path):
    data = []

    # PDF 열기
    with pdfplumber.open(pdf_path) as pdf:
        # 각 페이지 순회
        for page in pdf.pages:
            # 페이지에서 표 추출
            tables = page.extract_tables()

            # 각 표를 순회
            for table in tables:
                # 각 행을 데이터로 추가
                for row in table:
                    print(row)
                    if len(row) == 1 or 4 or 7:  # 열 개수가 맞는 경우에만 추가
                        data.append({
                            "단어": row[0].strip(),
                            "단어 뜻": row[3].strip(),
                            "예문": row[6].strip(),
                        })
    
    # JSON 파일로 저장
    with open("output.json", "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

    return data

# PDF 파일 경로
pdf_path = "./start.pdf"
parsed_data = pdf_to_json(pdf_path)

# 결과 출력
print(parsed_data)


['', '단어', '', '', '단어 뜻', '', '', '예문 (예문 뜻)', '']
['album', None, None, '사진첩', None, None, 'I have an album. 나는 사진첩 한 권이 있어.', None, None]
['ball', None, None, '공', None, None, 'I have a ball. 나는 공 하나가 있어.', None, None]
['bat', None, None, '야구 방망이', None, None, 'I want a bat. 나는 야구 방망이 하나를 원해.', None, None]
['book', None, None, '책', None, None, 'I have a book. 나는 책 한 권이 있어.', None, None]
['box', None, None, '상자', None, None, 'I want a box. 나는 상자 하나를 원해.', None, None]
['doll', None, None, '인형', None, None, 'I have a doll. 나는 인형 하나가 있어.', None, None]
['key', None, None, '열쇠', None, None, 'I want a key. 나는 열쇠 하나를 원해.', None, None]
['robot', None, None, '로봇', None, None, 'I want a robot. 나는 로봇 하나를 원해.', None, None]
['umbrella', None, None, '우산', None, None, 'I want an umbrella. 나는 우산 하나를 원해.', None, None]
['watch', None, None, '시계', None, None, 'I have a watch. 나는 시계 하나가 있어.', None, None]
['', '단어', '', '', '단어 뜻', '', '', '예문 (예문 뜻)', '']
['bear', None, None, '곰', None, None, 'Look at t

In [8]:
import pdfplumber
import pandas as pd
import json
import os
import re

def pdf_to_json(pdf_path, output_path):
    data = []

    # PDF 열기
    with pdfplumber.open(pdf_path) as pdf:
        # 각 페이지 순회
        for page in pdf.pages:
            # 페이지에서 표 추출
            tables = page.extract_tables()

            # 각 표를 순회
            for table in tables:
                # 각 행을 데이터로 추가
                for row in table:
                    print(row)
                    if len(row):  # 열 개수가 맞는 경우에만 추가
                        # 예문과 뜻을 나누기
                        example_sentence, example_meaning = "", ""
                        # 영어와 한국어 문장이 섞여 있을 때 분리 (이름 포함)
                        if re.search(r'[A-Za-z]+.*?[.!?]\s+[가-힣]', row[6]):
                            parts = re.split(r'(?<=[.!?])\s+(?=[가-힣])', row[6], 1)
                            example_sentence = parts[0].strip()
                            example_meaning = parts[1].strip() if len(parts) > 1 else ""
                        elif "\n" in row[6]:
                            parts = row[6].split("\n", 1)
                            example_sentence = parts[0].strip()
                            example_meaning = parts[1].strip()
                        else:
                            example_sentence = row[6].strip()
                            example_meaning = ""

                        # 개행 문자 제거 (예문 및 예문 뜻 열에서)
                        example_sentence = example_sentence.replace("\n", " ").replace("\r", " ")
                        example_meaning = example_meaning.replace("\n", " ").replace("\r", " ")

                        data.append({
                            "단어": row[0].strip(),
                            "단어 뜻": row[3].strip(),
                            "예문": example_sentence,
                            "예문 뜻": example_meaning,
                        })
    
    # 빈 딕셔너리 제거
    data = [entry for entry in data if any(entry.values())]

    # JSON 파일로 저장
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

    return data

# PDF 파일이 있는 디렉토리
pdf_directory = "./pdf_words"
output_directory = "./json_words"

# 디렉토리가 존재하지 않으면 생성
os.makedirs(output_directory, exist_ok=True)

# 각 PDF 파일에 대해 JSON 파일 생성
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        output_path = os.path.join(output_directory, pdf_file.replace(".pdf", ".json"))
        parsed_data = pdf_to_json(pdf_path, output_path)

        # 결과 출력
        print(f"Parsed data for {pdf_file}:", parsed_data)


['', '단어', '', '', '단어 뜻', '', '', '예문 (예문 뜻)', '']
['bird', None, None, '새', None, None, 'A bird is in the tree.\n새가 나무에 있다.', None, None]
['idea', None, None, '생각', None, None, 'Matt has a good idea.\n맷에게 좋은 생각이 있다.', None, None]
['wing', None, None, '날개', None, None, 'Chickens have wings.\n닭은 날개를 가지고 있다.', None, None]
['fly', None, None, '날다', None, None, 'Spiderman can’t fly.\n스파이더맨은 날 수 없다.', None, None]
['wind', None, None, '바람', None, None, 'The wind was strong.\n바람이 강했다.', None, None]
['car', None, None, '자동차', None, None, 'Is this your car?\n이것은 네 차니?', None, None]
['sky', None, None, '하늘', None, None, 'Look at the sky.\n하늘을 봐.', None, None]
['airplane', None, None, '비행기', None, None, 'The airplane is in the sky.\n비행기가 하늘에 있다.', None, None]
['', '단어', '', '', '단어 뜻', '', '', '예문 (예문 뜻)', '']
['cousin', None, None, '사촌', None, None, 'My cousins are kind.\n내 사촌들은 친절하다.', None, None]
['notebook', None, None, '공책', None, None, 'Do you have a new notebook?\n너는 새 공책이 있니?', None, Non

In [31]:
import pdfplumber
import pandas as pd
import json
import os
import re

def pdf_to_json(pdf_path, output_path):
    data = []

    # PDF 열기
    with pdfplumber.open(pdf_path) as pdf:
        # 각 페이지 순회
        for page in pdf.pages:
            # 페이지에서 표 추출
            tables = page.extract_tables()

            # 각 표를 순회
            for table in tables:
                # 각 행을 데이터로 추가
                for row in table:
                    print(row)
                    if len(row) > 0:  # 행이 비어있지 않은 경우에만 추가
                        # 예문과 뜻을 나누기
                        example_sentence, example_meaning = "", ""
                        # 영어와 한국어 문장이 섞여 있을 때 분리 (이름 포함)
                        if re.search(r'[A-Za-z]+.*?[.!?]\s+[가-힣]', row[6]):
                            parts = re.split(r'(?<=[.!?])\s+(?=[가-힣])', row[6], 1)
                            example_sentence = parts[0].strip()
                            example_meaning = parts[1].strip() if len(parts) > 1 else ""
                        elif re.search(r'[A-Za-z]+\s+[가-힣]', row[6]):  # 영어 이름 뒤에 한국어 문장이 오는 경우
                            parts = re.split(r'(?<=[A-Za-z])\s+(?=[가-힣])', row[6], 1)
                            example_sentence = parts[0].strip()
                            example_meaning = parts[1].strip() if len(parts) > 1 else ""
                        elif "\n" in row[6]:
                            parts = row[6].split("\n", 1)
                            example_sentence = parts[0].strip()
                            example_meaning = parts[1].strip()
                        else:
                            example_sentence = row[6].strip()
                            example_meaning = ""

                        # 개행 문자 제거 (예문 및 예문 뜻 열에서)
                        example_sentence = example_sentence.replace("\n", " ").replace("\r", " ")
                        example_meaning = example_meaning.replace("\n", " ").replace("\r", " ")

                        data.append({
                            "단어": row[0].strip(),
                            "단어 뜻": row[3].strip(),
                            "예문": example_sentence,
                            "예문 뜻": example_meaning,
                        })
    
    # 빈 딕셔너리 제거
    data = [entry for entry in data if any(entry.values())]

    # JSON 파일로 저장
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

    return data

# PDF 파일이 있는 디렉토리
pdf_directory = "./pdf_words"
output_directory = "./json_words"

# 디렉토리가 존재하지 않으면 생성
os.makedirs(output_directory, exist_ok=True)

# 각 PDF 파일에 대해 JSON 파일 생성
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        output_path = os.path.join(output_directory, pdf_file.replace(".pdf", ".json"))
        parsed_data = pdf_to_json(pdf_path, output_path)

        # 결과 출력
        print(f"Parsed data for {pdf_file}:", parsed_data)


['', '단어', '', '', '단어 뜻', '', '', '예문 (예문 뜻)', '']
['bird', None, None, '새', None, None, 'A bird is in the tree.\n새가 나무에 있다.', None, None]
['idea', None, None, '생각', None, None, 'Matt has a good idea.\n맷에게 좋은 생각이 있다.', None, None]
['wing', None, None, '날개', None, None, 'Chickens have wings.\n닭은 날개를 가지고 있다.', None, None]
['fly', None, None, '날다', None, None, 'Spiderman can’t fly.\n스파이더맨은 날 수 없다.', None, None]
['wind', None, None, '바람', None, None, 'The wind was strong.\n바람이 강했다.', None, None]
['car', None, None, '자동차', None, None, 'Is this your car?\n이것은 네 차니?', None, None]
['sky', None, None, '하늘', None, None, 'Look at the sky.\n하늘을 봐.', None, None]
['airplane', None, None, '비행기', None, None, 'The airplane is in the sky.\n비행기가 하늘에 있다.', None, None]
['', '단어', '', '', '단어 뜻', '', '', '예문 (예문 뜻)', '']
['cousin', None, None, '사촌', None, None, 'My cousins are kind.\n내 사촌들은 친절하다.', None, None]
['notebook', None, None, '공책', None, None, 'Do you have a new notebook?\n너는 새 공책이 있니?', None, Non

In [191]:
import pdfplumber
import pandas as pd
import json
import os
import re


import re

def extract_data_from_table(row):
    # 예문과 뜻을 초기화합니다.
    example_sentence, example_meaning = "", ""

    # 영어 문장 뒤에 한글 문장이 오고, 문장 끝에 마침표, 느낌표, 또는 물음표가 있는 경우 분리합니다.
    if re.search(r'[A-Za-z]+.*?[.!?]\s+[가-힣]', row[6]):
        parts = re.split(r'(?<=[.!?])\s+(?=[가-힣])', row[6], 1)
    # 영어 단어 뒤에 공백을 두고 바로 한글 문장이 시작되는 경우를 분리합니다.
    elif re.search(r'[A-Za-z]+\s+[가-힣]', row[6]):
        parts = re.split(r'(?<=[A-Za-z])\s+(?=[가-힣])', row[6], 1)
    # 줄 바꿈 문자로 구분된 영어 문장과 한글 뜻을 분리합니다.
    elif "\n" in row[6]:
        parts = row[6].split("\n", 1)
    else:
        parts = [row[6].strip(), ""]

    example_sentence = remove_newlines(parts[0].strip())
    mixed = extract_sentences_with_korean(example_sentence)
    example_meaning = remove_newlines(parts[1].strip()) if len(parts) > 1 else ""
    if example_meaning.startswith('씨'):
        concat = example_sentence + example_meaning
        example_meaning = extract_sentences_with_korean(concat)[0]
        example_sentence = concat.replace(example_meaning, '')
    if len(mixed) > 0:
        if mixed[0] in example_sentence:
            example_sentence = example_sentence.replace(mixed[0], '')
            example_meaning = mixed[0] + ' ' + example_meaning

    return {
        "단어": remove_newlines(row[0].strip()), 
        "단어 뜻": remove_newlines(row[3].strip()),
        "예문": example_sentence.strip(),
        "예문 뜻": example_meaning.strip(),
    }
    

def extract_sentences_with_korean(text):
    # 문장을 분리하는 정규 표현식
    sentences = re.split(r'(?<=[.?!])\s+', text)
    
    # 한글과 영어가 모두 포함된 문장만 필터링, 공백 포함 처리
    korean_sentences = [sentence for sentence in sentences if re.search(r'[가-힣]', sentence)]

    
    return korean_sentences



def pdf_to_json(pdf_path, output_path):
    data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    if len(row) > 0:
                        entry = extract_data_from_table(row)
                        if any(entry.values()):
                            data.append(entry)

    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

    return data

def process_pdfs(pdf_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    for pdf_file in os.listdir(pdf_directory):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, pdf_file)
            output_path = os.path.join(output_directory, pdf_file.replace(".pdf", ".json"))
            parsed_data = pdf_to_json(pdf_path, output_path)
            print(f"Parsed data for {pdf_file}:", parsed_data)

# 디렉토리 설정
pdf_directory = "./pdf_words"
output_directory = "./json_words"

# PDF 파일 처리
process_pdfs(pdf_directory, output_directory)

Parsed data for fly1.pdf: [{'단어': 'bird', '단어 뜻': '새', '예문': 'A bird is in the tree.', '예문 뜻': '새가 나무에 있다.'}, {'단어': 'idea', '단어 뜻': '생각', '예문': 'Matt has a good idea.', '예문 뜻': '맷에게 좋은 생각이 있다.'}, {'단어': 'wing', '단어 뜻': '날개', '예문': 'Chickens have wings.', '예문 뜻': '닭은 날개를 가지고 있다.'}, {'단어': 'fly', '단어 뜻': '날다', '예문': 'Spiderman can’t fly.', '예문 뜻': '스파이더맨은 날 수 없다.'}, {'단어': 'wind', '단어 뜻': '바람', '예문': 'The wind was strong.', '예문 뜻': '바람이 강했다.'}, {'단어': 'car', '단어 뜻': '자동차', '예문': 'Is this your car?', '예문 뜻': '이것은 네 차니?'}, {'단어': 'sky', '단어 뜻': '하늘', '예문': 'Look at the sky.', '예문 뜻': '하늘을 봐.'}, {'단어': 'airplane', '단어 뜻': '비행기', '예문': 'The airplane is in the sky.', '예문 뜻': '비행기가 하늘에 있다.'}, {'단어': 'cousin', '단어 뜻': '사촌', '예문': 'My cousins are kind.', '예문 뜻': '내 사촌들은 친절하다.'}, {'단어': 'notebook', '단어 뜻': '공책', '예문': 'Do you have a new notebook?', '예문 뜻': '너는 새 공책이 있니?'}, {'단어': 'vacation', '단어 뜻': '방학', '예문': 'I like summer vacation.', '예문 뜻': '나는 여름 방학이 좋다.'}, {'단어': 'come', '단어 뜻': '오다', '예문

In [186]:
def extract_sentences_with_korean(text):
    # 문장을 분리하는 정규 표현식
    sentences = re.split(r'(?<=[.?!])\s+', text)
    
    # 한글과 영어가 모두 포함된 문장만 필터링, 공백 포함 처리
    korean_sentences = [sentence for sentence in sentences if re.search(r'[가-힣]', sentence)]

    
    return korean_sentences


text = '''
You must brush your hair every day. 
너는  매일  머리를  빗어야  해.
'''

extract_sentences_with_korean(text)

['너는  매일  머리를  빗어야  해.']